<a href="https://colab.research.google.com/github/txrds/kaggle_Titanic/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [169]:
df = pd.read_csv('/content/drive/MyDrive/Projetos/Titanic/titanic/train.csv')

In [170]:
test_df = pd.read_csv('/content/drive/MyDrive/Projetos/Titanic/titanic/test.csv')

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
test_df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
df['Ticket'].unique()

In [ ]:
df['Ticket'].value_counts()

In [143]:
moda_age = df['Age'].mode()[0]
media_age = df['Age'].mean()
mediana_age = df['Age'].median()

In [144]:
mediana_age

28.0

### In the provided dataset, missing values in the 'Age' column were replaced by the median, while missing values in the 'Cabin' column were replaced by the mode.

## traind.csv

In [171]:
# Replaced missing values with the median
df['Age'] = df['Age'].fillna(mediana_age if pd.isnull(mediana_age) else mediana_age)

In [172]:
# Replace missing values with mode
df['Cabin'] = df['Cabin'].fillna(df['Cabin'].mode()[0])

In [173]:
# Replace missing values with mode
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

In [174]:
# change categorical data to numerical data
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
df['Cabin'] = le.fit_transform(df['Cabin'])
df['Embarked'] = le.fit_transform(df['Embarked'])

## test.csv

In [175]:
# change 'test_df'categorical data to numerical data
le = LabelEncoder()
test_df['Sex'] = le.fit_transform(test_df['Sex'])
test_df['Cabin'] = le.fit_transform(test_df['Cabin'])
test_df['Embarked'] = le.fit_transform(test_df['Embarked'])

In [176]:
# Replaced missing values with the median
test_df['Age'] = test_df['Age'].fillna(mediana_age if pd.isnull(mediana_age) else mediana_age)
# Replace missing values with mode
test_df['Cabin'] = test_df['Cabin'].fillna(test_df['Cabin'].mode()[0])
# Replace missing values with mode
test_df['Embarked'] = test_df['Embarked'].fillna(test_df['Embarked'].mode()[0])
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].mode()[0])

## regressão logística

In [167]:
X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']]  # Features
y = df['Survived']  # Variável alvo

In [156]:
# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [157]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [158]:
# Criando e treinando o modelo de regressão logística
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [159]:
y_pred = model.predict(X_test)

In [160]:
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

Acurácia: 0.8100558659217877

Matriz de Confusão:
[[90 15]
 [19 55]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179



## Arvore de Decisão

In [100]:
# Criar e treinar o modelo de árvore de decisão
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

Acurácia: 0.7597765363128491

Matriz de Confusão:
[[82 23]
 [20 54]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.80      0.78      0.79       105
           1       0.70      0.73      0.72        74

    accuracy                           0.76       179
   macro avg       0.75      0.76      0.75       179
weighted avg       0.76      0.76      0.76       179



##Florestas Aleatórias (Random Forest)

In [102]:
# Criar e treinar o modelo de Floresta Aleatória
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

Acurácia: 0.7932960893854749

Matriz de Confusão:
[[90 15]
 [22 52]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.80      0.86      0.83       105
           1       0.78      0.70      0.74        74

    accuracy                           0.79       179
   macro avg       0.79      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179



## Máquinas de Vetores de Suporte (SVM)

In [182]:
X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']]
y = df['Survived']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronizar as features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Criar e treinar o modelo SVM
model = SVC(random_state=42)
model.fit(X_train_scaled, y_train)

# Fazer previsões
y_pred = model.predict(X_test_scaled)

# Avaliar o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:")

Acurácia: 0.8324022346368715

Matriz de Confusão:
[[92 13]
 [17 57]]

Relatório de Classificação:


In [179]:
X_test = test_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']]

In [180]:
predictions = model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [181]:
# Criar DataFrame com as previsões
submission_df = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': predictions
})

# Salvar as previsões em um arquivo CSV
submission_df.to_csv('submission.csv', index=False)
